In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  


from keras.applications.nasnet import NASNetLarge
from keras.preprocessing import image
from keras import backend as K
from keras.models import Model, load_model
import numpy as np
from keras.utils import to_categorical
import os
import glob
import cv2
import tensorflow as tf
from scipy.io import loadmat, savemat
from keras.applications.nasnet import preprocess_input
from datetime import datetime

import efficientnet.keras as efn 





Using TensorFlow backend.
c:\users\zys61\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\zys61\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\zys61\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
model = efn.EfficientNetB7(weights='noisy-student')  # or weights='noisy-student'

# model = NASNetLarge(input_shape=(331, 331, 3), include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000)
model.summary()

W1017 17:26:36.704929   860 deprecation_wrapper.py:119] From c:\users\zys61\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1017 17:26:36.722881   860 deprecation_wrapper.py:119] From c:\users\zys61\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1017 17:26:36.726871   860 deprecation_wrapper.py:119] From c:\users\zys61\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4479: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W1017 17:26:36.741831   860 deprecation_wrapper.py:119] From c:\users\zys61\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:190: The name tf.get_default_sessio

Model: "efficientnet-b7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 600, 600, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 300, 300, 64) 1728        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 300, 300, 64) 256         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 300, 300, 64) 0           stem_bn[0][0]                    
____________________________________________________________________________________

__________________________________________________________________________________________________
block7a_project_conv (Conv2D)   (None, 19, 19, 640)  1474560     block7a_se_excite[0][0]          
__________________________________________________________________________________________________
block7a_project_bn (BatchNormal (None, 19, 19, 640)  2560        block7a_project_conv[0][0]       
__________________________________________________________________________________________________
block7b_expand_conv (Conv2D)    (None, 19, 19, 3840) 2457600     block7a_project_bn[0][0]         
__________________________________________________________________________________________________
block7b_expand_bn (BatchNormali (None, 19, 19, 3840) 15360       block7b_expand_conv[0][0]        
__________________________________________________________________________________________________
block7b_expand_activation (Acti (None, 19, 19, 3840) 0           block7b_expand_bn[0][0]          
__________

In [3]:
model_cut = Model(inputs=model.inputs, output=model.layers[-3].output)

c:\users\zys61\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("av...)`
  """Entry point for launching an IPython kernel.


In [4]:
datasets=['Office_caltech_10','Office31','Office_home'];
str_domains_Office_caltech_10 = ['caltech', 'amazon', 'webcam', 'dslr']; 
str_domains_Office31 = ['amazon', 'webcam', 'dslr']; 
str_domains_Office_home = ['Art', 'Clipart', 'Product', 'Real_World']; 

k=0

for ii in range(len(eval('str_domains_'+datasets[k]))):  
    print('Feature extraction on ' + eval('str_domains_'+datasets[k])[ii])
    X_train = [] # images as arrays
    Y_train = [] # labels

    alldata_dir = os.path.join(os.getcwd(), eval('str_domains_'+datasets[k])[ii])

    Folders =  os.listdir(alldata_dir)
    class_name2id = { label: index for index, label in enumerate(Folders) } # Map class names to int
    for folder in Folders:
        for filename in os.listdir(os.path.join(alldata_dir,folder)):
            img = cv2.imread(os.path.join(os.path.join(alldata_dir,folder),filename))
            if img is not None:
                img = cv2.resize(img, (600, 600))
                X_train.append(img)
                Y_train.append(class_name2id.get(folder))

    X_train= np.array(X_train)
    Y_train= np.array(Y_train)
    print(X_train.shape)
    features = model_cut.predict(preprocess_input(X_train), batch_size=4, verbose=0)
    
    savemat('Python_efficientnet_'+eval('str_domains_'+datasets[k])[ii]+'.mat', {'features':features, 'labels':Y_train+1})

Feature extraction on caltech
(1123, 600, 600, 3)
Feature extraction on amazon
(958, 600, 600, 3)
Feature extraction on webcam
(295, 600, 600, 3)
Feature extraction on dslr
(157, 600, 600, 3)
